In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
# Loading the data

train_data =  pd.read_csv('../input/demand-forecasting-kernels-only/train.csv',parse_dates=True, index_col =0)
#train_data2 = train_data
#train_data2
y_train = train_data.iloc[:,-1].values
train_data.drop('sales', 1, inplace=True)
train_data['y'] = train_data.index.year-train_data.index.year.min()
train_data['m'] = train_data.index.month
train_data['d'] = train_data.index.day
train_data['dow'] = train_data.index.dayofweek


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_data, y_train, test_size=.1, random_state=0, shuffle = True)

In [ ]:
x_train

In [ ]:
cat_vars = list(train_data.columns)
cat_vars.remove('y')
cont_vars = ['y']
#cont_vars

In [ ]:
X_train = []
X_val = []
X_train.append(x_train[cont_vars].astype('float32').values)
X_val.append(x_val[cont_vars].astype('float32').values)
for cat in cat_vars:
    X_train.append(x_train[cat].values)
    X_val.append(x_val[cat].values)


In [ ]:
cat_sizes = {}
cat_embsizes = {}
for cat in cat_vars:
    cat_sizes[cat] = train_data[cat].nunique()
    cat_embsizes[cat] = min(50, cat_sizes[cat]//2+1)
#cat_sizes

In [ ]:
cat_embsizes

In [ ]:
test_data = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv', parse_dates=True, index_col =1)
test_data['y'] = test_data.index.year-train_data.index.year.min()
test_data['m'] = test_data.index.month
test_data['d'] = test_data.index.day
test_data['dow'] = test_data.index.dayofweek
# test_data['special_store'] = test_data['store'].isin([5,6,7])*1
#test_data['y']
#test_data.index.year

In [ ]:
X_test = []
X_test.append(test_data[cont_vars].astype('float32').values)
for cat in cat_vars:
    X_test.append(test_data[cat].values)

In [ ]:
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate
from keras.models import Model

In [ ]:
import keras.backend as K

def custom_smape(x, x_):
    return K.mean(2*K.abs(x-x_)/(K.abs(x)+K.abs(x_)))

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
y = Input( shape = (len(cont_vars),), name='cont_vars')
ins = [y]
concat = [y]
for cat in cat_vars:
    x = Input((1,), name=cat)
    ins.append(x)
    x = Embedding(cat_sizes[cat]+1, cat_embsizes[cat], input_length=1)(x)
    x = Reshape((cat_embsizes[cat],))(x)
    concat.append(x)
y = Concatenate()(concat)
y = Dense(100, activation= 'relu')(y)
# y = Dense(100, activation= 'relu')(y)
y = Dense(1)(y)
model = Model(ins, y)
model.summary()
model.compile('adadelta', custom_smape)



In [ ]:
#model.fit(X_train, y_train, 64, 2,  validation_data=(X_val, y_val)) #validation_data=[X_val, y_val])
model.fit(X_train, y_train, 64, 2,  validation_data=[X_val, y_val])

In [ ]:
test_preds = model.predict(X_test)
sample_data = pd.read_csv("../input/demand-forecasting-kernels-only/sample_submission.csv")
sample_data['sales'] = test_preds
sample_data.to_csv('preds.csv')